# Treasure Island

## Import packages

In [1]:
import numpy as np
from numpy import typing as npt

from typing import List, Tuple

In [2]:
h_func = lambda x, y: -x/y * np.log2(x/y)

In [3]:
3 / 5 * (h_func(2, 3) + h_func(1, 3))

0.5509775004326937

In [4]:
1 - 0.551

0.44899999999999995

In [5]:
sigmoid = lambda x: 1 / (1 + np.exp(-x)) 
h1 = 1 / (1 + np.exp(4))
h2 = 1 / (1 + np.exp(0))

In [6]:
sigmoid(0.3 * h1 + 0.6 * h2)

0.5757610475392924

In [32]:
a = np.array([[0, 1, 1],
              [1, 2, 2]])
b = np.array([1, 2])
b.astype()

/tmp/ipykernel_3066/2404730261.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  a in b


False

### Set a Random Number Generator

In [6]:
rng = np.random.RandomState(42)

In [5]:
class UserInterface:
    pass

In [6]:
class MapGenerator:
    def __init__(self) -> None:
        pass

In [2]:
class Map:
    def __init__(
        self, 
        W: int, 
        H: int,
        value: np.ndarray,                                  # matrix of tiles's value (string)
        region: np.ndarray,                                 # matrix of tiles's region (int)
        scanned: np.ndarray,                                # matrix of tiles that is scanned (bool)
        potential: np.ndarray,                              # matrix of tiles potential (bool)
        agent: Tuple[int, int],                             # agent's coordinate
        pirate: Tuple[int, int],                            # pirate's coordinate
        treasure: Tuple[int, int],                          # treasure's coordinate
        total_region: int,                                  # maximum number of regions
        regions: List[List[Tuple[int, int]]],               # tiles that categorize based on regions (1, 2, ..., max region)
        beaches: List[List[Tuple[int, int]]],               # tiles that are at the seaside
        boundaries: List[List[Tuple[int, int]]]             # tiles that are in boundary between two regions
    ) -> None:
        self.total_tile = W * H
        self.shape = (W, H)
        self.value = value 
        self.region = region
        self.scanned = scanned
        self.potential = potential
        self.agent = agent
        self.pirate = pirate
        self.treasure = treasure
        self.total_region = total_region
        self.regions = regions
        self.beaches = beaches
        self.boundaries = boundaries

        # Map generate hints function to string
        self.hints = {"1": self.generate_hint_1, "2": self.generate_hint_2, "3": self.generate_hint_3, "4": self.generate_hint_4,
                      "5": self.generate_hint_5, "6": self.generate_hint_6, "7": self.generate_hint_7, "8": self.generate_hint_8,
                      "9": self.generate_hint_9, "10": self.generate_hint_10, "11": self.generate_hint_11, "12": self.generate_hint_12,
                      "13": self.generate_hint_13, "14": self.generate_hint_14, "15": self.generate_hint_15, "16": self.generate_hint_16}

    def hint_generator(self):
        np.random.randint(16)

    def ravel_index(self, index: Tuple[int, int]) -> int:
        W, H = self.shape
        return H * index[0] + index[1]

    
    def generate_hint(self) -> None:
        self.hints[str(rng.randint(16))]()
    
    def generate_hint_1(self) -> Tuple[bool, List[Tuple[int, int]], str]:
        # A list of random tiles that doesn't contain the treasure (1 to 12)

        # trueness of this hint
        trueness = True

        # get random tiles doest not contain the treasure
        no_tiles = rng.randint(1, 13)
        rand_tiles = rng.choice(np.arange(self.total_tile), size=no_tiles, replace=False)

        # get tile that overlaps with the treasure
        overlap = rand_tiles == self.ravel_index(self.treasure)

        # coordinate of these random tiles
        tile_coords = np.unravel_index(rand_tiles, self.shape)

        # if one of them contain the treasure
        if overlap.any():
            trueness = False
            self.potential[tile_coords] = True
            masked_tiles = np.ones(self.shape, dtype=bool)
            masked_tiles[tile_coords] = False
            self.scanned[masked_tiles] = True
        
        # if they are not contain the treasure
        else:
            self.scanned[tile_coords] = True

        hinted_tiles = list(zip(tile_coords[0], tile_coords[1]))

        log = f"These tiles {hinted_tiles} do not contain the treasure"
                        
        return trueness, hinted_tiles, log
        
    def generate_hint_2(self) -> Tuple[bool, List[int], str]:
        # 2-5 regions that 1 of them has the treasure.

        # trueness of this hint
        trueness = False

        # number of regions
        no_reg = rng.randint(1, 5)
        rand_regions = rng.choice(np.arange(1, self.total_region + 1), size=no_reg, replace=False)
        
        # get region that overlaps with the treasure's region
        overlap = rand_regions == self.region[self.treasure]

        # get mask of titles of those regions
        masked_tiles = np.isin(self.region, rand_regions)

        # if random region consist of a region that has the treasure
        if overlap.any():
            trueness = True
            self.potential[masked_tiles] = True
            self.scanned[~masked_tiles] = True

        # if they are not contain the treasure
        else:
            self.scanned[masked_tiles] = True
            
        hinted_regions = list(rand_regions)
        
        log = f"One of these regions contain the treasure: {hinted_regions}"
            
        return trueness, hinted_regions, log

    def generate_hint_3(self) -> Tuple[bool, List[int], str]:
        # 1-3 regions that do not contain the treasure.

        # trueness of this hint
        trueness = True

        # number of regions
        no_reg = rng.randint(1, 3)
        rand_regions = rng.choice(np.arange(1, self.total_region + 1), size=no_reg, replace=False)

        # get region that overlaps with the treasure's region
        overlap = rand_regions == self.region[self.treasure]

        # get mask of titles of those regions
        masked_tiles = np.isin(self.region, rand_regions)

        # if random region consist of a region that has the treasure
        if overlap.any():
            trueness = False
            self.potential[masked_tiles] = True
            self.scanned[~masked_tiles] = True

        # if they are not contain the treasure
        else:
            self.scanned[masked_tiles] = True
            
        hinted_regions = list(rand_regions)
        
        log = f"These regions do not contain the treasure: {hinted_regions}"

        return trueness, list(hinted_regions), log


    def generate_hint_4(self) -> Tuple[bool, Tuple[Tuple[int, int], Tuple[int, int]], str]:
        # A large rectangle area that has the treasure

        trueness = False
        w_size = int(0.5*self.shape[0])
        h_size = int(0.5*self.shape[1])
        
        start_point_x = rng.randint(0, self.shape[0] - w_size + 1)
        start_point_y = rng.randint(0, self.shape[1] - h_size + 1)
        
        end_point_x = start_point_x + w_size - 1
        end_point_y = start_point_y + h_size - 1
        
        if start_point_x <= self.treasure[1] <= end_point_x and start_point_y <= self.treasure[0] <= end_point_y:
            trueness = True
            self.potential[start_point_y:end_point_y+1][start_point_x:end_point_x+1] = True
            masked_tiles = np.ones(self.shape, dtype=bool)
            masked_tiles[start_point_y:end_point_y+1][start_point_x:end_point_x+1] = False
            self.scanned = np.logical_or(self.scanned, masked_tiles)
        else:
            trueness = False
            self.scanned[start_point_y:end_point_y+1][start_point_x:end_point_x+1] = True
            
        top_left = (start_point_y, start_point_x)
        bottom_right = (end_point_y, end_point_x)
        
        hinted_coord = (top_left, bottom_right)
        
        log = f"Large rectangle area has the treasure. Top-Left-Bottom-Right = [{start_point_y}, {start_point_x}, {end_point_y}, {end_point_x}]"
        
        return trueness, hinted_coord, log

    def generate_hint_5(self) -> Tuple[bool, Tuple[Tuple[int, int], Tuple[int, int]], str]:
        # A small rectangle area that doesn't has the treasure.

        trueness = False
        w_size = int(0.2*self.shape[0]) ape[0] - w_size + 1)
        start_point_y = rng.randint(0, self.shape[1] - h_size + 1)
        
        end_point_x = start_point_x + w_size - 1
        end_point_y = start_point_y + h_size - 1
        
        if start_point_x <= self.treasure[1] <= end_point_x and start_point_y <= self.treasure[0] <= end_point_y:
            trueness = False
            self.potential[start_point_y:end_point_y+1][start_point_x:end_point_x+1] = True
            masked_tiles = np.ones(self.shape, dtype=bool)
            masked_tiles[start_point_y:end_point_y+1][start_point_x:end_point_x+1] = False
            self.scanned = np.logical_or(self.scanned, masked_tiles)
        else:
            trueness = True
            self.scanned[start_point_y:end_point_y+1][start_point_x:end_point_x+1] = True
            
        top_left = (start_point_y, start_point_x)
        bottom_right = (end_point_y, end_point_x)
        
        hinted_coord = (top_left, bottom_right)
        
        log = f"Small rectangle area doesn't the treasure. Top-Left-Bottom-Right = [{start_point_y}, {start_point_x}, {end_point_y}, {end_point_x}]"
        
        return trueness, hinted_coord, log

    def generate_hint_6(self) -> Tuple[bool, None, str]:
        # You are the nearest person to the treasure

        # calculate the distances
        agent_treasure = abs(sum(self.agent) ** 2 - sum(self.treasure) ** 2)
        pirate_treasure = abs(sum(self.pirate) ** 2 - sum(self.treasure) ** 2)

        # trueness of this hint
        trueness = agent_treasure > pirate_treasure

        log = "You are the nearest person to the treasure"

        return trueness, None, log

    def generate_hint_7(self) -> List[Cell]:
        # A column and/or a row that contain the treasure (rare)

        pass

    def generate_hint_8(self) -> List[Cell]:
        # A column and/or a row that do not contain the treasure

        pass

    def generate_hint_9(self) -> List[Cell]:
        # 2 regions that the treasure is somewhere in their boundary

        pass

    def generate_hint_10(self) -> List[Cell]:
        # The treasure is somewhere in a boundary of 2 regions

        pass

    def generate_hint_11(self) -> List[Cell]:
        # The treasure is somewhere in an area bounded by 2-3 tiles from sea
 
        pass

    def generate_hint_12(self) -> Tuple[bool, int, str]:
        # A half of the map without treasure

        # trueness of this hint
        trueness = rng.randint(2, dtype=bool)

        # random part of the map (0: left, 2: top, 3: bottom, 4: right)
        parts = ["left", "top", "bottom", "right"]
        part = rng.randint(4)

        match part:
            case 0:
                vertical_middle_axis = (self.shape[1] - 1) // 2 + 1
                if trueness:
                    self.scanned[:, :vertical_middle_axis] = True
                else:
                    self.potential[:, :vertical_middle_axis] = True
                    self.scanned[:, vertical_middle_axis:] = True

            case 1:
                horizontal_middle_axis = (self.shape[0] - 1) // 2 + 1
                if trueness:
                    self.scanned[:horizontal_middle_axis] = True
                else:
                    self.potential[:horizontal_middle_axis] = True
                    self.scanned[horizontal_middle_axis:] = False

            case 2:
                horizontal_middle_axis = (self.shape[0] + 1) // 2
                if trueness:
                    self.scanned[horizontal_middle_axis:] = True
                else:
                    self.potential[horizontal_middle_axis] = True
                    self.scanned[:horizontal_middle_axis] = False

            case 3:
                vertical_middle_axis = (self.shape[1] - 1) // 2
                if trueness:
                    self.scanned[:, vertical_middle_axis:] = True
                else:
                    self.potential[:, vertical_middle_axis:] = True
                    self.scanned[:, :vertical_middle_axis] = True
        
        log = f"{parts[part]} part of the map does not contain the treasure."

        return trueness, part, log

    def generate_hint_13(self) -> List[Cell]:
        # From the center of the map/from the prison that he's staying, he tells
        # you a direction that has the treasure (W, E, N, S or SE, SW, NE, NW)

        pass

    def generate_hint_14(self) -> List[Cell]:
        # 2 squares that are different in size, the small one is placed inside the
        # bigger one, the treasure is somewhere inside the gap between 2 squares
        
        pass

    def generate_hint_15(self) -> List[Cell]:
        # The treasure is in a region that has mountain

        pass

    def generate_hint_16(self) -> List[Cell]:
        pass

SyntaxError: invalid syntax (777433636.py, line 95)

In [7]:
7 // 2

3

In [ ]:
class Agent:
    def __init_(self) -> None:
        self.position = Cell()
        
        pass

In [ ]:
class JackSparrow(Agent):
    def __init__(self) -> None:
        pass
    
    def action(self):
        pass

    def move(self, step: int) -> None:
        pass

    def verifies_hint(self) -> bool:
        pass

    def small_scan(self) -> None:
        pass

    def large_scan(self) -> None:
        pass

    def teleport(self) -> None:
        pass

    def is_treasure(self) -> bool:
        pass

In [ ]:
class Hint:
    def __init__(self, trueness: bool) -> None:
        self.trueness = trueness

        pass



In [ ]:
class Pirate(Agent):
    def __init__(self) -> None:
        super().__init__()
        path = self.shorted_path()
        hints = []
        pass

    def shorted_path(self) -> List[Tuple[int, str]]:
        pass

    def gives_hint(self) -> None:
        pass

In [103]:
import numpy as np
Y = np.random.randint(2, size=(4, 4))
X = np.ones(Y.shape, dtype=int)
print(Y)
x_1, y_1 = 0, 1
x_2, y_2 = 2, 2
X[x_1:x_2+1, y_1:y_2+1] = 0
Y = np.logical_or(Y, X)
print(Y)

[[1 0 0 1]
 [0 0 1 0]
 [1 1 0 1]
 [1 1 1 1]]
[[ True False False  True]
 [ True False  True  True]
 [ True  True False  True]
 [ True  True  True  True]]
